# Customer Segmentation Clustering Exercise
This notebook demonstrates K-Means, K-Means++, Agglomerative, and Divisive clustering on a real customer dataset.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from collections import Counter

## Load Dataset

In [ ]:
from io import StringIO

csv_data = StringIO("""CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
1,Male,19,15,39
2,Male,21,15,81
3,Female,20,16,6
4,Female,23,16,77
5,Female,31,17,40
6,Female,22,17,76
7,Female,35,18,6
8,Female,23,18,94
9,Male,64,19,3
10,Female,30,19,72
""")

df = pd.read_csv(csv_data)
df.head()

### Encode & Standardize

In [ ]:
# Convert gender to numeric
df['Gender'] = df['Gender'].map({'Male':0, 'Female':1})

# Select features
X = df[['Gender','Age','Annual Income (k$)','Spending Score (1-100)']].values

# Standardize
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

print("Data shape:", Xs.shape)

## Determine Optimal k using Elbow & Silhouette

In [ ]:
ks = range(2,11)
inertia = []
sil_scores = []

for k in ks:
    km = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=42)
    labels = km.fit_predict(Xs)
    inertia.append(km.inertia_)
    sil_scores.append(silhouette_score(Xs, labels))

plt.figure()
plt.plot(ks, inertia, marker='o')
plt.title("Elbow Method")
plt.xlabel("k")
plt.ylabel("Inertia")
plt.grid(True)
plt.show()

plt.figure()
plt.plot(ks, sil_scores, marker='o')
plt.title("Silhouette Score")
plt.xlabel("k")
plt.ylabel("Silhouette")
plt.grid(True)
plt.show()

best_k = ks[sil_scores.index(max(sil_scores))]
best_k

## Clustering Models Comparison

In [ ]:
# Final KMeans++
km_final = KMeans(n_clusters=best_k, init='k-means++', n_init=20, random_state=42)
labels_km = km_final.fit_predict(Xs)

# Agglomerative
agg = AgglomerativeClustering(n_clusters=best_k)
labels_agg = agg.fit_predict(Xs)

# Divisive clustering
def divisive_kmeans(X, k_target):
    clusters = {0: np.arange(X.shape[0])}
    while len(clusters) < k_target:
        cid = max(clusters, key=lambda c: np.sum((X[clusters[c]] - X[clusters[c]].mean(0))**2))
        idx = clusters.pop(cid)
        km2 = KMeans(n_clusters=2, n_init=10, random_state=42)
        sub = km2.fit_predict(X[idx])
        clusters[cid] = idx[sub==0]
        clusters[len(clusters)] = idx[sub==1]
    labels = np.zeros(X.shape[0],dtype=int)
    for i,c in enumerate(clusters.values()):
        labels[c] = i
    return labels

labels_div = divisive_kmeans(Xs, best_k)

# Silhouette comparison
print("Silhouette Scores:")
print("KMeans++:", silhouette_score(Xs, labels_km))
print("Agglomerative:", silhouette_score(Xs, labels_agg))
print("Divisive:", silhouette_score(Xs, labels_div))